<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [6]:
def u2i_eval_daily_summary(rec_daily, mapping_tab, rec_num):
    a = rec_daily.groupby('user_ID').apply(lambda x: x.sort_values(by='request_time')['sku_ID'].tolist())
    
    ## main part
    pred_cover_by_user = a.to_frame().apply(lambda x: pd.Series(eval_helper(mapping_tab=mapping_tab, click_list=x[0], rec_num=rec_num, user_ID=x.name)), axis=1)
    
    pred_suc, total_pred_num = pred_cover_by_user.sum(axis=0)
    pred_suc_ratio = pred_suc/total_pred_num
    return pred_suc_ratio, pred_suc, total_pred_num, pred_cover_by_user

def get_daily_formated_summary(rec_daily, ui2i_tab, tech_type, recommend_type, date):
    res_list = []
    for rec_num in range(1,10):
        if recommend_type == "user to item":
            pred_suc_ratio, pred_suc, total_pred_num, pred_cover_by_user = u2i_eval_daily_summary(rec_daily, mapping_tab=ui2i_tab, rec_num=rec_num)
        elif recommend_type == "item to item":
            pred_suc_ratio, pred_suc, total_pred_num, pred_cover_by_user = i2i_eval_daily_summary(rec_daily, mapping_tab=ui2i_tab, rec_num=rec_num)
        # print(pred_suc_ratio)
        res_list.append(["rec_num = " + str(rec_num), pred_suc_ratio])

    res_list.append(["total_click_samples", total_pred_num])
    res_list.append(["method", str(tech_type) + ' && '+ str(recommend_type)])
    daily_summary = pd.DataFrame.from_dict(dict(res_list), orient='index')
    daily_summary.columns = ['suc_ratio_on_'+str(date)]
    return daily_summary, pred_cover_by_user

In [14]:
import pandas as pd 

rec_daily = pd.read_csv('rec_daily.csv', index_col=0)[['sku_ID', 'user_ID', 'request_time']]
i2i_tab = pd.read_csv("i2i_tab.csv", index_col=0)
u2i_tab = pd.read_csv("u2i_tab.csv", index_col=0)
w2v_tab = pd.read_csv("w2v_tab.csv", index_col=0)


In [36]:
date, tech_type, recommend_type = now, "w2v based recommendation", "item to item"
# ui2i_tab = daily_routing.load_middle(date, tech_type, recommend_type)
ui2i_tab = w2v_tab.apply(lambda x: pd.Series(eval(x[0])), axis=1)
ds5, _ = get_daily_formated_summary(rec_daily, ui2i_tab, tech_type, recommend_type, date)


In [37]:
ds5

,suc_ratio_on_2018-03-14
rec_num = 1,0.751457
rec_num = 2,0.772655
rec_num = 3,0.788553
rec_num = 4,0.794913
rec_num = 5,0.804981
rec_num = 6,0.811341
rec_num = 7,0.824059
rec_num = 8,0.827769
rec_num = 9,0.840488
total_click_samples,1887


In [16]:
u2i_tab

,0,1,2,3,4,5,6,7,8,9
001058d384,df66d5239a,932d4c1646,e99eb7d131,8d643cf7bf,d2b980826a,d20f9b641c,d10b8d2fba,d0567f9df7,294371350c,ce96007f29
0019f28911,81e57cbc50,d50c683162,51da628bf9,a25b6de145,29e4f19c34,9f8c6352c1,d775432144,d92261201a,9ce99a29c2,9baa8fffc1
001a59814c,81e57cbc50,d10b8d2fba,baf3fbb0be,436e59bbcb,bc1fef9128,c4ff8911d9,38676d45f5,ccceb52cc9,35674a6b48,35333da584
002057bd3a,7261e762c4,ab4dacb110,b5e2de2bc1,b4841fb3f7,1e98111ec9,1e4d2db819,46c288b53e,af94cd1455,ac61f4e10e,48c26fa27d
002c086940,81e57cbc50,2f9e750498,067b673f2b,75cdd218b9,48c26fa27d,ac61f4e10e,61fd12c10d,89f3796b30,51da628bf9,f16794cf8a
...,...,...,...,...,...,...,...,...,...,...
ffef0d2590,81e57cbc50,ac61f4e10e,b378989140,c1827caab1,c4c37a4b57,38676d45f5,c4ff8911d9,c8ce38c12f,35674a6b48,3418d59e22
fff084489f,81e57cbc50,ccceb52cc9,5710927122,d10b8d2fba,d0567f9df7,29c8797899,29e4f19c34,ce15d3c844,8b930b0131,2b82f7ac49
fff5d94a76,81e57cbc50,aff785a666,3dd5460585,3ef4723684,b720d538ec,b4ac267fa7,436e59bbcb,43be829eb9,b378989140,443346ceab
fff6126d6c,81e57cbc50,d775432144,30751566fe,a887d53803,32118a2170,a8c80a32be,5f413aa088,d7c786790b,d70cd7d343,38676d45f5


,suc_ratio_on_2018-03-14,suc_ratio_on_2018-03-14
rec_num = 1,0.048755,0.048755
rec_num = 2,0.067303,0.067303
rec_num = 3,0.085321,0.085321
rec_num = 4,0.101219,0.101219
rec_num = 5,0.108638,0.108638
rec_num = 6,0.120827,0.120827
rec_num = 7,0.144144,0.144144
rec_num = 8,0.144674,0.144674
rec_num = 9,0.147324,0.147324
total_click_samples,1887,1887


In [ ]:
def 

In [5]:
daily_summary

,suc_ratio_on_2018-03-14
rec_num = 1,0.048755
rec_num = 2,0.067303
rec_num = 3,0.085321
rec_num = 4,0.101219
rec_num = 5,0.108638
rec_num = 6,0.120827
rec_num = 7,0.144144
rec_num = 8,0.144674
rec_num = 9,0.147324
total_click_samples,1887
